<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [1]:
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:
#!unzip concrete_data_week4.zip

In [3]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
#from keras.applications import ResNet50
#from keras.applications.resnet import preprocess_input, ResNet50
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np


In [4]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [5]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [7]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [8]:
model_vgg = Sequential()

In [9]:
model_vgg.add(VGG16(include_top=False, pooling='avg', weights='imagenet' ))

In [10]:
model_vgg.add(Dense(num_classes, activation='softmax'))

In [11]:
model_vgg.layers
model_vgg.layers[0].layers

In [12]:
model_vgg.layers[0].trainable = False

In [13]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [16]:
# C:~\Temp\ipykernel_5236\783624917.py:1:
# UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. 
# Please use `Model.fit`, which supports generators. <- I need to use the corrected method, but I don't want to wait for training again
# fit_history = model_vgg.fit_generator 

fit_history = model_vgg.fit_generator(
   train_generator,
   steps_per_epoch=steps_per_epoch_training,
   epochs=num_epochs,
   validation_data=validation_generator,
   validation_steps=steps_per_epoch_validation,
   verbose=1,
   shuffle=False
)

C:\Users\vadim\AppData\Local\Temp\ipykernel_5236\783624917.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model_vgg.fit_generator(


Epoch 1/2
301/301 [==============================] - 2421s 8s/step - loss: 0.1129 - accuracy: 0.9608 - val_loss: 0.0256 - val_accuracy: 0.9941
Epoch 2/2
301/301 [==============================] - 2370s 8s/step - loss: 0.0189 - accuracy: 0.9954 - val_loss: 0.0155 - val_accuracy: 0.9959


In [ ]:
# =======For the convenience of verification, I will save the learning results as text=============
# Epoch 1/2
# 301/301 [==============================] - 2421s 8s/step - loss: 0.1129 - 
# accuracy: 0.9608 - val_loss: 0.0256 - val_accuracy: 0.9941

# Epoch 2/2
# 301/301 [==============================] - 2370s 8s/step - loss: 0.0189 - 
# accuracy: 0.9954 - val_loss: 0.0155 - val_accuracy: 0.9959

In [25]:
model_vgg_saved = model_vgg.save('classifier_vgg16.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [61]:
model_resnet50 = keras.models.load_model('classifier_resnet_model.h5')
model_vgg_saved = keras.models.load_model('classifier_vgg16.h5')



In [62]:
testing_generator = data_generator.flow_from_directory(
    'concrete_data_week4/concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical', shuffle=False)

Found 500 images belonging to 2 classes.


In [63]:
performance_vgg = model_vgg_saved.evaluate(testing_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg[0], 5)))
print("Accuracy: {}".format(round(performance_vgg[1], 5)))


5/5 [==============================] - 30s 6s/step - loss: 0.0135 - accuracy: 0.9960
Performance of the VGG16-trained model
Loss: 0.01353
Accuracy: 0.996


In [64]:
performance_resnet50 = model_resnet50.evaluate(testing_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

5/5 [==============================] - 13s 2s/step - loss: 0.0019 - accuracy: 1.0000
Performance of the ResNet50-trained model
Loss: 0.00188
Accuracy: 1.0


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [65]:
prediction_vgg = model_vgg.predict(testing_generator, steps = 1)

def print_res_predict(prediction):
    if prediction[0] > prediction[1]:
        print(f"Predict is Negative {round(prediction[0]*100,2)}% guaranty")
    elif prediction[0] < prediction[1]:
        print(f"Predict is Positive {round(prediction[1]*100,2)}% guaranty")
    else:
        print("Result not guaranteed")

1/1 [==============================] - 6s 6s/step


In [66]:
print("===== Predictions for the VGG16 model, for the 5 examples: =====")
print()
for i in range(5):
    
    print_res_predict(prediction_vgg[i])

===== Predictions for the VGG16 model, for the 5 examples: =====

Predict is Negative 99.68% guaranty
Predict is Negative 96.7% guaranty
Predict is Negative 97.92% guaranty
Predict is Negative 99.64% guaranty
Predict is Negative 98.87% guaranty


In [67]:
predictions_resnet = model_resnet50.predict(testing_generator, steps=1)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_res_predict(predictions_resnet[i])


1/1 [==============================] - 3s 3s/step
First five predictions for the ResNet50-trained model:
Predict is Negative 99.98% guaranty
Predict is Negative 99.93% guaranty
Predict is Negative 99.86% guaranty
Predict is Negative 99.93% guaranty
Predict is Negative 99.93% guaranty


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).